In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('../data/covid_econonomic_factors.xlsx', sheet_name = [
    'gdp_growth_rate', 'gdp_per_cap_PPP', 'personal_income_tax_rate',
    'personal_income_tax_rate', 'debt_to_gdp', 'ease_of_doing_business',
    'internet_speed', 'unemployment_rate', 'consumer_price_index', 
    'inflation_rates', 'coronavirus_cases', 'coronavirus_deaths', 
    'coronavirus_recovered', 'coronavirus_vaccination_rate', 'coronavirus_vaccination_total',
    'corruption_rank', 'population', 'gini_index'
])

edu_factors = pd.read_excel('../data/edu_factors.xlsx', sheet_name=[
    'gov_exp_on_edu_percent_of_gdp', 'gov_exp_on_edu_perc_per_student',
    'gov_exp_per_stud_tertiary_perc', 'pupil_teacher_ratio_primary',
    'youth_literacy_rate'
])

continent = pd.read_csv('../data/countryContinent.csv')

In [2]:
len(df.keys())

17

In [3]:
final_df = pd.DataFrame(df['gdp_growth_rate'][['Country', 'Last']])
final_df.columns = ['Country', 'gdp_growth_rate']
for i in df.keys():
    df_tmp = df[i][['Country', 'Last']]
    df_tmp.columns = ['Country', i]
    final_df = final_df.merge(df_tmp)

In [4]:
final_df.columns

Index(['Country', 'gdp_growth_rate', 'gdp_per_cap_PPP',
       'personal_income_tax_rate', 'debt_to_gdp', 'ease_of_doing_business',
       'internet_speed', 'unemployment_rate', 'consumer_price_index',
       'inflation_rates', 'coronavirus_cases', 'coronavirus_deaths',
       'coronavirus_recovered', 'coronavirus_vaccination_rate',
       'coronavirus_vaccination_total', 'corruption_rank', 'population',
       'gini_index'],
      dtype='object')

In [5]:
final_df['covid_death_rate'] = final_df['coronavirus_deaths']/final_df['population']
final_df['covid_cases_rate'] = final_df['coronavirus_cases']/final_df['population']
final_df['covid_recovered'] = final_df['coronavirus_recovered']/final_df['population']
final_df = final_df.rename(columns={'coronavirus_vaccination_rate':'covid_vaccination_rate'})

In [6]:
final_df = continent[['country', 'continent', 'sub_region']].merge(final_df.rename(columns={'Country':'country'}), on='country')

In [7]:
final_edu = pd.DataFrame(edu_factors['gov_exp_on_edu_percent_of_gdp'][['Country', 'Most Recent Value']])
final_edu.columns = ['Country', 'gov_exp_on_edu_percent_of_gdp']
for i in edu_factors.keys():
    df_tmp = edu_factors[i][['Country', 'Most Recent Value']]
    df_tmp.columns = ['Country', i]
    final_edu = final_edu.merge(df_tmp)

final_edu = final_edu.rename(columns={'Country':'country'})

In [8]:
final_edu.country = np.where(final_edu.country == 'Bahamas, The', 'Bahamas', 
         np.where(final_edu.country == "Cote d'Ivoire", 'Ivory Coast',
         np.where(final_edu.country == "Egypt, Arab Rep.", 'Egypt',
         np.where(final_edu.country == "Gambia, The", 'Gambia',
         np.where(final_edu.country == "Hong Kong SAR, China", 'Hong Kong',
         np.where(final_edu.country == "Korea, Rep.", "South Korea",
         np.where(final_edu.country == "North Macedonia", 'Macedonia',
         np.where(final_edu.country == "Russian Federation", "Russia",
         np.where(final_edu.country == "Slovak Republic", "Slovakia", 
         final_edu.country)))))))))

In [9]:
for i in final_df.country.unique():
    if i not in final_edu.country.unique():
        print(i)

In [10]:
final_df = final_edu.merge(final_df, on='country')

In [11]:
final_df

,country,gov_exp_on_edu_percent_of_gdp,gov_exp_on_edu_perc_per_student,gov_exp_per_stud_tertiary_perc,pupil_teacher_ratio_primary,youth_literacy_rate,continent,sub_region,gdp_growth_rate,gdp_per_cap_PPP,...,coronavirus_deaths,coronavirus_recovered,covid_vaccination_rate,coronavirus_vaccination_total,corruption_rank,population,gini_index,covid_death_rate,covid_cases_rate,covid_recovered
0,Afghanistan,3.2,11.3,58.2,49.0,47.0,Asia,Southern Asia,-1.93,1979,...,7307,69205,12.84,5116767,165,33.20,NaN,220.090361,4733.253012,2084.487952
1,Albania,3.9,8.0,13.8,18.0,99.0,Europe,Southern Europe,17.90,13295,...,3077,130009,72.46,2081782,104,2.85,33.2,1079.649123,69576.140351,45617.192982
2,Algeria,6.1,17.9,NaN,24.0,97.0,Africa,Northern Africa,6.40,10682,...,6041,97089,26.97,12032500,104,44.23,27.6,136.581506,4739.407642,2195.093828
3,Angola,1.8,NaN,82.1,50.0,77.0,Africa,Middle Africa,1.20,6198,...,1732,33242,26.60,9026310,142,33.00,51.3,52.484848,1972.878788,1007.333333
4,Argentina,4.8,20.6,16.9,17.0,100.0,Americas,South America,17.90,19687,...,116505,4062245,147.00,67104574,78,45.38,42.9,2567.320405,117321.264874,89516.196562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,United Arab Emirates,3.1,8.1,0.0,25.0,98.0,Asia,Western Asia,-6.10,67119,...,2145,611442,216.00,21548528,21,9.90,26.0,216.666667,74921.212121,61761.818182
102,United Kingdom,5.2,21.2,38.0,15.0,NaN,Europe,Northern Europe,6.60,41627,...,144593,344,167.00,113910449,11,67.20,35.1,2151.681548,149863.705357,5.119048
103,United States,NaN,22.1,19.4,14.0,NaN,Americas,Northern America,4.90,60236,...,775797,16306656,135.00,454447737,25,329.00,41.4,2358.045593,146281.379939,49564.303951
104,Vietnam,4.1,NaN,34.2,20.0,99.0,Asia,South-Eastern Asia,-6.17,8200,...,24407,6764,115.00,112944634,104,97.58,35.7,250.122976,11972.002460,69.317483


In [12]:
final_df.to_csv('../data/final_df.csv', index=False)